> PyTorch is first and foremost a deep learning library.

To do deep learning, need to 
* ingest data
* define the model
* train the model 


PyTorch uses the core data structure `torch.tensor`, which is similar to the `numpy.ndarray`, and features accelerated mathematical operations. These can be used on the CPU or the GPU. Moving to GPU can be done in a few function calls (allegedly).

PyTorch keeps track of operaations performed on `tensor`s, and this can be used to analytically compute derivatives with respect to any inputs. This greatly simplifies numerical optimizations. This opens PyTorch up to many scientific disciplines, not only machine learning (see physics, optimization, simulation, modeling).



In [10]:
import torch
print(f"torch version:  {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

torch version:  1.6.0
CUDA available: False


## Pretrained network example

In [19]:
from torchvision import models
dir(models)[0:5]

alexnet = models.AlexNet()

`alexnet` is an object which contains the AlexNet architecture.

In [21]:
print(alexnet)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

We can push data through this network, but we will get garbage since this is, of course, untrained. We can traain it or load models.

In [24]:
resnet = models.resnet101(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth" to /Users/programming/.cache/torch/hub/checkpoints/resnet101-5d3b4d8f.pth
100%|██████████| 170M/170M [00:04<00:00, 36.5MB/s]


The building blocks of a neural network are `torch.nn.Module`s. They are individual layers, or individual operations. We can print our model to see the `modules`, one per line:

In [94]:
resnet;

We can pass data to the model by calling the model like a function.

First, we need to pre-process the inputs. For images, we need to standardize both their size and their intensity values.

`torchvision.transforms` contains many basic preprocessing functions.

In [31]:
from torchvision import transforms
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [105]:
from PIL import Image
img = Image.open("./data/man.jpg")

In [106]:
img_t = preprocess(img)
img_t.shape

torch.Size([3, 224, 224])

In [107]:
batch_t = torch.unsqueeze(img_t, 0)

In [108]:
batch_t.shape

torch.Size([1, 3, 224, 224])

In [109]:
# put network into eval mode, to do inference
resnet.eval();

In [110]:
out = resnet(batch_t)

In [111]:
with open("./data/imagenet_classes.txt") as fin:
    labels = [line.strip() for line in fin.readlines()]

In [112]:
_, index = torch.max(out,1)
labels[index]

'sweatshirt'

In [113]:
percentage = torch.nn.functional.softmax(out, dim=1)
percentage = percentage[0] * 100
labels[index[0]], percentage[index[0]].item()

('sweatshirt', 99.03517150878906)

In [114]:
_, indices = torch.sort(out,descending=True)
[(labels[idx], percentage[idx].item()) for idx in indices[0][:5]]

[('sweatshirt', 99.03517150878906),
 ('lab coat, laboratory coat', 0.4077696204185486),
 ('ski mask', 0.06618981808423996),
 ('gar, garfish, garpike, billfish, Lepisosteus osseus', 0.044084902852773666),
 ('thunder snake, worm snake, Carphophis amoenus', 0.028538847342133522)]